# Retrieve player statistics 
and convert them into player profiles (offensive, defensive, neutral). It also shows how to retrieve game info, such as players list etc.


In [1]:
from ReinforcementLearning.NHL.playerstats.nhl_player_stats import *
from ReinforcementLearning.NHL.playbyplay.game import *

In [2]:
# Pointers to the data
# repoCode    =   '/Users/younes_zerouali/Documents/Stradigi/Code/NHL_stats_SL'
repoCode    =   '/Users/luisd/dev/NHL_stats'
#db_root     =   '/home/younesz/Documents/Hockey'        #This is the location of the Hockey database
# db_root     =   '/Users/younes_zerouali/Documents/Stradigi/Databases/Hockey'
db_root     =   '/Users/luisd/dev/NHL_stats/data'
repoPbP     =   path.join(db_root, 'PlayByPlay')
repoPSt     =   path.join(db_root, 'PlayerStats/player')
repoModel   =   path.join(repoCode, 'ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_backup_trainedonallseasons_rankstatprediction')
repoModel   =   path.join(repoCode, 'ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu')
repoSave    =   None #path.join(repoCode, 'ReinforcementLearning/NHL/playbyplay/data')

In [3]:
repoPbP

'/Users/luisd/dev/NHL_stats/data/PlayByPlay'

In [4]:
repoModel

'/Users/luisd/dev/NHL_stats/ReinforcementLearning/NHL/playerstats/offVSdef/Automatic_classification/MODEL_perceptron_1layer_10units_relu'

# Re-train the model if needed
For example if you have trouble because the file of the model is not accessible, you need to re-run this cell.

In [5]:
# ==== Before starting, recompute the model
# This should avoid path issues when reloading it
# normalizer, pca, dtCols, CLS    =   do_ANN_training(repoPSt, repoPbP, repoCode, repoModel, minGames=0.2)

# Definitions of Functions

## Get game id's by team and date

In [6]:
def get_game_id(home_team_abbr, date_as_str):
    """
    let's convert game date to game code.
    For example Montreal received Ottawa on march 13, 2013 =>
        gameId = get_game_id(home_team_abbr='MTL', date_as_str='2013-03-13')
    """
    try:
        gameInfo    =   pickle.load( open(path.join(db_root, 'gamesInfo.p'), 'rb') )
        gameInfo    =   gameInfo[gameInfo['gameDate']==date_as_str][gameInfo['teamAbbrev']==home_team_abbr]
        gameId      =   gameInfo['gameId']
        gameId      =   int( gameId.values.astype('str')[0][5:] )
        return gameId
    except Exception as e:
        raise IndexError("There was no game for '%s' on '%s'" % (home_team_abbr, date_as_str))

## Get players data (classes)

In [7]:
# ==== Second the players data

def get_players_classes(repoModel, data_for_game, number_of_games):
    """
    Calculates (dataframe with) all player's classes.
    Updates the 'data for game' structure with it; also returns it.
    Usage:
        repoModel = ... # here goes the directory where your model is saved.
        # Montreal received Ottawa on march 13, 2013
        gameId = get_game_id(home_team_abbr='MTL', date_as_str='2013-03-13')
        season      =   '20122013'
        mtlott      =   Game(repoPbP, repoPSt, season, gameId=gameId )
        #
        players_classes = get_players_classes(repoModel, mtlott, number_of_games=30)
        # this is equivalent to ask 'mtlott' for the data; so:
        assert players_classes.equals(mtlott.player_classes)
    """
    # Need to load the data pre-processing variables
    preprocessing  =   pickle.load(open(path.join(repoModel, 'baseVariables.p'), 'rb'))

    # Need to load the classification model (for players' predicted ranking on trophies voting lists)
    classifier          =   {'sess':tf.Session(), 'annX':[], 'annY':[]}
    saver               =   tf.train.import_meta_graph(path.join(repoModel, path.basename(repoModel) + '.meta'))
    graph               =   classifier['sess'].graph
    classifier['annX']  =   graph.get_tensor_by_name('Input_to_the_network-player_features:0')
    classifier['annY']  =   graph.get_tensor_by_name('prediction:0')
    saver.restore(classifier['sess'], tf.train.latest_checkpoint(path.join(repoModel, './')))

    # Pick players stats - last 'n' games 
    data_for_game.pull_line_shifts(team='both', minduration=20)
    data_for_game.pick_regulartime()
    data_for_game.pick_equalstrength()
    data_for_game.pull_players_classes(preprocessing, classifier, nGames=number_of_games)
    return data_for_game.player_classes

# Let's run analysis for MTL-OTT on March 13, 2013

In [8]:
# ==== First the game data

# Montreal received Ottawa on march 13, 2013, let's convert game date to game code
gameId = get_game_id(home_team_abbr='MTL', date_as_str='2013-03-13')

# Now lets get game data 
season      =   '20122013'
mtlott      =   Game(repoPbP, repoPSt, season, gameId=gameId )

/Users/luisd/sw/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [9]:
# ==== Second the players data

# Visualize all player's classes: 0=def, 1=off, 2=neutral
players_classes = get_players_classes(repoModel, data_for_game=mtlott, number_of_games=30)
assert players_classes.equals(mtlott.player_classes)
mtlott.player_classes


/Users/luisd/sw/anaconda3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator PCA from version 0.19.1 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


,class,firstlast,pred_ross,pred_selke
8,0,DAVID DESHARNAIS,0.065531,0.398755
9,0,LARS ELLER,0.071325,0.467515
13,2,ALEX GALCHENYUK,0.191740,0.248879
14,1,BRENDAN GALLAGHER,0.436872,0.202157
27,1,MAX PACIORETTY,0.380084,0.353909
31,1,TOMAS PLEKANEC,0.338640,0.383409
228,2,CHRIS NEIL,0.121128,0.318634
236,0,ZACK SMITH,0.003851,0.450651
237,2,MARK STONE,0.057223,0.328079
238,2,KYLE TURRIS,0.186008,0.412824


In [10]:
# === Now we get the indices in the Q-values tables corresponding to lines

# Line translation table
linedict  = HockeySS(repoPbP, repoPSt)
linedict.make_line_dictionary()
linedict  = linedict.line_dictionary

# Get lines and translate them 
playersCode  =   mtlott.encode_line_players()
linesCode    =   np.array( [[mtlott.recode_line(linedict, a) for a in b] for b in playersCode] )


In [16]:
# Visualize the line codes for each shift (first digit is the code of the away team's line - state-space, 
# second digit is the code of the home team's line - action space)
linesCode

array([[ 2,  5],
       [ 4,  4],
       [ 8,  5],
       [ 2,  5],
       [ 4,  5],
       [ 4,  5],
       [ 8,  5],
       [ 8,  5],
       [ 2,  5],
       [ 4,  6],
       [ 8,  5],
       [ 4,  5],
       [-1,  6],
       [ 8,  5],
       [ 4,  5],
       [ 7,  5],
       [ 2,  5],
       [ 4,  5],
       [ 8,  5],
       [ 7,  4],
       [ 2,  5],
       [ 4,  5],
       [ 8,  5],
       [ 4,  5],
       [ 2,  5],
       [ 4,  5],
       [ 8,  5],
       [ 8,  5],
       [ 2,  5],
       [ 4,  5],
       [ 8,  5],
       [ 2,  5],
       [ 4,  5],
       [ 4,  5],
       [ 8,  5],
       [ 2, -1],
       [ 8,  5],
       [ 2,  5],
       [ 4,  5],
       [ 2,  5],
       [ 2,  5],
       [ 2,  5],
       [-1,  6],
       [ 2, -1],
       [ 4,  5],
       [-1,  6],
       [ 7,  4],
       [ 2,  5],
       [ 8,  6],
       [ 2,  4],
       [ 4,  5],
       [ 4,  5],
       [ 8,  6],
       [ 2,  5],
       [ 5,  4],
       [ 8,  5],
       [ 2,  6]])

In [18]:
# ==== Now we want to evaluate the value of the first shift for the (home team)

# Load the Qvalues table
Qvalues = pickle.load( open(path.join(repoCode, 'ReinforcementLearning/NHL/playbyplay/data/stable/RL_action_values.p'), 'rb') )['action_values']

# Visualize it dimensions (period x differential x away line's code x home line's code)
print('Q-table dimensions: ', Qvalues.shape)

# Get the Q-value for that specific line
iShift = 0 # First shift
plList = list(mtlott.player_classes.loc[mtlott.lineShifts['playersID'].iloc[iShift][0]]['firstlast'].values) + list(mtlott.player_classes.loc[mtlott.lineShifts['playersID'].iloc[iShift][1]]['firstlast'].values)
diff   = mtlott.recode_differential( mtlott.lineShifts.iloc[iShift].differential )
period = mtlott.recode_period( mtlott.lineShifts.iloc[iShift].period )
q_values = Qvalues[period,diff,linesCode[iShift,0], linesCode[iShift,1]]
print('[diff = %d, period = %d] First shift: \n\thome team: %s, %s, %s \n\taway team: %s, %s, %s \n\tQvalue: %.2f' %(diff, period, plList[0], plList[1], plList[2], plList[3], plList[4], plList[5], q_values) )
                                                                          


Q-table dimensions:  (3, 5, 10, 10)
[diff = 2, period = 0] First shift: 
	home team: DAVID DESHARNAIS, BRENDAN GALLAGHER, MAX PACIORETTY 
	away team: KYLE TURRIS, JAKOB SILFVERBERG, COLIN GREENING 
	Qvalue: 2.77
